# HPSS Report Emailer

This notebook ingests an HPSS Daily Report text, creates graphical representations of some records, and crafts an HTML email with the plots embedded.

In [ ]:
%matplotlib inline

In [ ]:
import datetime
import copy
import re
import os

In [ ]:
import matplotlib
import matplotlib.pyplot
import tokio

In [ ]:
# Input parameters for the notebook
TARGET_DATE = datetime.datetime(2020, 3, 13)
TARGET_SYSTEM = 'archive'

EXAMPLE_REPORT_SECTION = 'io totals by hpss client gateway (ui) host'

In [ ]:
# Identify the text file containing the daily report from flanders
report_filename = tokio.tools.common.enumerate_dated_files(
    start=TARGET_DATE,
    end=TARGET_DATE + datetime.timedelta(days=1),
    template=tokio.config.CONFIG.get('hpss_report_files'))[0]

# Convert that text file into a Python dictionary
hpss_report = tokio.connectors.hpss.HpssDailyReport(report_filename)

In [ ]:
# Show an example of the structure of a single record section
hpss_report[TARGET_SYSTEM][EXAMPLE_REPORT_SECTION]['heart'].keys()

In [ ]:
def plot_report_section(report_section, read_key, write_key, include_total=False, ax=None):
    """Plots a read-vs-write horizontal bar graph of a report section
    
    Args:
        report_section (dict): A key for for the HpssDailyReport[SYSTEM]
            dictionary.  Something like ``io totals by hpss client gateway (ui) host``
        read_key (str): Column corresponding to a read value (``read_gb`` or ``r_ops``)
        write_key (str): Column corresponding to a write value (``write_gb`` or ``w_ops``)
        include_total (bool): If True, include the sum of all rows as its own hbar
        ax (matplotlib.axes.Axes): If provided, axes in which plot should be added

    Returns:
        matplotlib.axes.Axes: Axes on which plot was added.
    """
    if ax is None:
        fig, ax = matplotlib.pyplot.subplots()

    yticklabels = []
    for idx, host in enumerate(report_section.keys()):
        if host == 'total' and not include_total:
            continue
        ax.barh(y=idx, width=report_section[host][write_key], color="C%d" % idx, edgecolor='black')
        ax.barh(y=idx, width=-report_section[host][read_key], color="C%d" % idx, edgecolor='black')
        yticklabels.append((idx, host))    

    # this needs to immediately follow the barh since it alters tick widths
    fig.set_size_inches(8.0, 2 + 6.0/35.0 * len(list(report_section.keys())))

    ax.grid() # or ax.xaxis.grid(True)
    ax.set_axisbelow(True)

    max_x = max([abs(x) for x in ax.get_xlim()])
    ax.set_xlim([-max_x, max_x])
    ax.set_xticklabels([int(abs(x)) for x in ax.get_xticks()], rotation=30)
    trans = matplotlib.transforms.blended_transform_factory(
        ax.transAxes,
        ax.transData)
    ax.text(x=0.0, y=idx - 0.5, s="Read", transform=trans, ha="left", va="top")
    ax.text(x=1.0, y=idx - 0.5, s="Write", transform=trans, ha="right", va="top")

    yticks, yticklabels = zip(*yticklabels)
    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels)
    ax.set_ylim(ax.get_ylim())

    ax.plot([0, 0], [-10, idx+10], color='black', linewidth=1)
    
    return ax

In [ ]:
# Show an example plot
plot_report_section(hpss_report[TARGET_SYSTEM][EXAMPLE_REPORT_SECTION], read_key="read_gb", write_key="write_gb")

In [ ]:
attach_files = []
for report_section in hpss_report[TARGET_SYSTEM].keys():
    # largest users is ordered, so it needs to be transformed into a dict
    if report_section == "largest users":
        copied_report_section = copy.deepcopy(hpss_report[TARGET_SYSTEM][report_section])
        flat_report_section = {}
        for record in copied_report_section:
            key = record.pop('user')
            flat_report_section[key] = record
        report_section_contents = flat_report_section
    else:
        report_section_contents = hpss_report[TARGET_SYSTEM][report_section]

    try:
        ax = plot_report_section(
            report_section=report_section_contents,
            read_key="read_gb",
            write_key="write_gb")
        ax.set_title("%s - %s" % (
            re.sub(r'^io totals ', '', report_section)\
                .title()\
                .replace("Hpss", "HPSS")\
                .replace("Ui", "UI"),
            TARGET_DATE.strftime("%b %d, %Y")))
        ax.set_xlabel("Total I/O (GB)")
        filename = "%s_%s_%s.png" % (
            TARGET_SYSTEM,
            re.sub(r'[\W]+', '', report_section),
            TARGET_DATE.strftime("%Y-%m-%d"))
        print("Saving output to %s"  % filename)
        attach_files.append(filename)
        ax.get_figure().savefig(filename, bbox_inches='tight', transparent=True)
    except (KeyError, AttributeError):
        continue

In [ ]:
# Send an HTML email with an embedded image and a plain text message for
# email clients that don't want to display the HTML.

from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

# Define these once; use them twice!
strFrom = os.environ.get('USER')
strTo = os.environ.get('USER')

# Create the root message and fill in the from, to, and subject headers
msgRoot = MIMEMultipart('related')
msgRoot['Subject'] = 'Daily HPSS Report - Graphical!'
msgRoot['From'] = strFrom
msgRoot['To'] = strTo
msgRoot.preamble = 'This is a multi-part message in MIME format.'

# Encapsulate the plain and HTML versions of the message body in an
# 'alternative' part, so message agents can decide which they want to display.
msgAlternative = MIMEMultipart('alternative')
msgRoot.attach(msgAlternative)

msgText = MIMEText('This is the alternative plain text message.')
msgAlternative.attach(msgText)

# We reference the image in the IMG SRC attribute by the ID we give it below
msg_html = "Daily report for %s:<br>" % TARGET_DATE.strftime("%Y-%m-%d")
for attach_file in attach_files:
    msg_html += '<img src="cid:%s" width="100%%"><br>' % attach_file
    
msgText = MIMEText(msg_html, 'html')
msgAlternative.attach(msgText)

for attach_file in attach_files:
    # This example assumes the image is in the current directory
    with open(attach_file, 'rb') as fp:
        msgImage = MIMEImage(fp.read())

    # Define the image's ID as referenced above
    msgImage.add_header('Content-ID', '<%s>' % attach_file)
    msgRoot.attach(msgImage)

# Send the email (this example assumes SMTP authentication is required)
import smtplib
smtp = smtplib.SMTP()
smtp.connect()
# smtp.login('exampleuser', 'examplepass')
smtp.sendmail(strFrom, strTo, msgRoot.as_string())
smtp.quit()